# <p style="text-align: center;"> Final Project - Introduction to Data Science</p>
# <p style="text-align: center;"> <b> Data Collecting </b></p>
---

## Member Information
| Name              | ID       |
|-------------------|----------|
|Tran Đinh Quang    | 21127406 |
|Nguyen Hong Hanh   | 21127503 |
|Do Quoc Tri  | 21127556 |
| Nguyen Khanh Nhan | 21127657         |


## Table of contents



---

## Import libaries

In [2]:
import requests
import sys
import json
import pandas as pd
from datetime import datetime, timedelta, date

## Data introduction

The data is fetched from the [Visualcrossings](https://ww.visualcrossing.com/) website. The data in this study pertains to the weather in Ho Chi Minh City from 01/01/2007 to 31/12/2022. Due to the ID code limitations allowing the retrieval of only 1000 queries of data per day, each run can obtain 1000 queries of data and record the date of the run into a file for subsequent runs to continue.

## Get the data

You can obtain a free account at Visualcrossings which comes with a key.

In [3]:
key = 'PEVKTTBLZUTL89Z6WHQ5V2HRK'
num_of_days = 999

with open('../data/lastdate.txt', 'r') as file:
  start = file.read()

end = datetime.strptime(start, '%Y-%m-%d') + timedelta(days=num_of_days)
end = end.strftime('%Y-%m-%d')

**Causion** The next cell will get the data from the website. As we can only get 1000 queries per day, **ONLY RUN IT ONCE**

In [4]:
response = requests.request("GET", f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Ho%20Chi%20Minh/{start}/{end}?unitGroup=us&include=days&key={key}&contentType=json")
if response.status_code!=200:
  print('Unexpected Status code: ', response.status_code)
  sys.exit()  

jsonData = response.json()

## Check the location infomation

In [5]:
location_info ={}
features = ['latitude', 'longitude', 'resolvedAddress', 'address', 'timezone', 'tzoffset']
for f in features:
  location_info[f] = jsonData[f]
location_info

{'latitude': 10.7764,
 'longitude': 106.701,
 'resolvedAddress': 'Quận 1, Hồ Chí Minh, Việt Nam',
 'address': 'Ho Chi Minh',
 'timezone': 'Asia/Ho_Chi_Minh',
 'tzoffset': 7.0}

## Get some data samples

In [6]:
df = pd.DataFrame.from_dict(jsonData['days'])
df.sample()

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source
208,2013-01-17,1358355600,88.9,66.2,78.3,88.8,66.2,78.5,62.5,60.5,...,06:16:10,1358378170,17:50:26,1358419826,0.18,"Rain, Partially cloudy",Partly cloudy throughout the day with late aft...,rain,"[48900099999, VVTS]",obs


## Save the data to file

In [7]:
df.to_csv(f'../data/raw_{start}_{end}.csv', index=False)

**Caution** The following cell will save the date for next time run. **Do not run next cell before finished**

In [8]:
end = datetime.strptime(end, '%Y-%m-%d') + timedelta(days=1)
end = end.strftime('%Y-%m-%d')
with open('../data/lastdate.txt', 'w') as file:
  file.write(end)